In [10]:
import os
from os.path import join
from time import time
from tqdm.notebook import tqdm

import numpy as np
from numpy import flatnonzero as find
from scipy.sparse import hstack, vstack
from scipy.sparse.linalg import spsolve

from data import case_as_data, PowerflowDataset


In [11]:
data = case_as_data('case300')
pf = PowerflowDataset(data)

$j$ denotes imaginary unit in electrical engineering  
$Y_{bus} = Y = G + jB$  
$S_{bus} = S = P + jQ$  
$V_i = |V_i|e^{j\delta_i}$  
$\theta_{ik} = \delta_i - \delta_k$

$S_k = V_k \cdot (YV)_k^*$  
or   
$P_{i}=\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\cos \theta _{{ik}}+B_{{ik}}\sin \theta _{{ik}})$    
$Q_{i}=\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\sin \theta _{{ik}}-B_{{ik}}\cos \theta _{{ik}})$  
Further reading: https://wiki.openelectrical.org/index.php?title=Power_Flow  
### Newton algorthm
$$
\Delta P_{i}=-P_{i}+\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\cos \theta _{{ik}}+B_{{ik}}\sin \theta _{{ik}})
\\
\Delta Q_{{i}}=-Q_{{i}}+\sum _{{k=1}}^{N}|V_{i}||V_{k}|(G_{{ik}}\sin \theta _{{ik}}-B_{{ik}}\cos \theta _{{ik}})
\\
J={\begin{bmatrix}{\dfrac  {\partial \Delta P}{\partial \theta }}&{\dfrac  {\partial \Delta P}{\partial |V|}}\\{\dfrac  {\partial \Delta Q}{\partial \theta }}&{\dfrac  {\partial \Delta Q}{\partial |V|}}\end{bmatrix}}
\\
{\begin{bmatrix}\Delta \theta \\\Delta |V|\end{bmatrix}}=-J^{{-1}}{\begin{bmatrix}\Delta P\\\Delta Q\end{bmatrix}}
\\
\theta ^{{m+1}}=\theta ^{m}+\Delta \theta \,
\\
|V|^{{m+1}}=|V|^{m}+\Delta |V|\,$$   
**What is not clarified yet**: types of buses: $pv$ and $pq$

In the power-flow problem, it is assumed that the real power $P_D$ and reactive power $Q_D$ at each Load Bus are known. For this reason, Load Buses are also known as **PQ Buses**. For Generator Buses, it is assumed that the real power generated $P_G$ and the voltage magnitude $|V|$ is known. For this reason, Generator Buses are also known as **PV Buses**

In [12]:
class NewtonRaphson():
    def __init__(self):
        pass
    def solve(self, regression, iterations=10, x0=None,
                 tol=1e-10, verbose=True):
        if x0 is None:
            # special case for PowerflowDataset
            if hasattr(regression, 'get_V0'): 
                x = regression.get_V0()
            else:
                x = np.random.rand(regression.A.shape[1])
        else:
            x = x0
        
        # initialize starting time
        start = time()
        # save the history of iterations for plotting and analysis
        x_history = [x]
        values_history = [np.linalg.norm(regression.forward(x), 2)]
        grads_history = [regression.grad(x)]
        times_history = [0]
        
        for i in tqdm(range(iterations)):
            if verbose:
                print(f'{np.linalg.norm(regression.forward(x), 2)}')
            J = regression.grad(x)
            F = regression.forward(x)
            dx = -spsolve(J, F) # - np.linalg.inv(J.toarray()) @ F
            x = regression.updatedV(x, dx)
            
            # update histories
            x_history.append(x)
            values_history.append(np.linalg.norm(regression.forward(x), 2))
            times_history.append(time() - start)
            
            if np.abs(values_history[-1] - values_history[-2]) <= tol:
                print(f'Newton-Raphson converged on iteration {i}')
                break
        return x_history, values_history, times_history
        

In [13]:
nr = NewtonRaphson()

res = list(nr.solve(pf, iterations=300, tol=1e-15, verbose=False))